In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

import os
import config

os.environ["OPENAI_API_KEY"]= config.openai_api_key

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
base_chain = prompt | model

# Exec into a docker container.
docker exec -it <container id> /bin/bash
# Use Redis CLI
redis-cli
# Look for all the available keys.
KEYS *
# To view the details of keys, use the below command.
LRANGE message_store:math-thread1 0 -1

In [ ]:
REDIS_URL = "redis://localhost:6379/0"
store = {}

In [ ]:
def get_message_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

redis_chain = RunnableWithMessageHistory(
    base_chain,
    get_message_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
redis_chain.invoke(
    {"ability": "math", 
     "input": "What does cosine mean?"}, 
    config={"configurable": {"session_id": "math-thread1"}},
)

In [ ]:
redis_chain.invoke(
    {"ability": "math", "input": "Tell me more!"},
    config={"configurable": {"session_id": "math-thread1"}},
)

In [ ]:
redis_chain.invoke(
    {"ability": "physics", 
     "input": "What is the theory of relativity?"}, 
    config={"configurable": {"session_id": "phy-thread1"}},
)

In [ ]:
redis_chain.invoke(
    {"ability": "physics", "input": "Tell me more!"},
    config={"configurable": {"session_id": "phy-thread1"}},
)